In [1]:
import project_utils as u
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary cache directory at /tmp/matplotlib-9chkbyst because the default path (/home/jovyan/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [5]:

FORM_990_EXTRACT_PATH = "./data/form_990/"
FORM_990_ZIP_PATH = "./load/form_990/"
FORM_990_PROCESSED_PATH = "./processed/form_990/"

CMS_YEAR = 2020
CMS_DATASET_ID = f"hospitals_10_2022"
CMS_URL = f'https://data.cms.gov/provider-data/sites/default/files/archive/Hospitals/{CMS_YEAR}/{CMS_DATASET_ID}.zip'
CMS_ZIP_PATH = f"./load/hospitals/"
CMS_EXTRACT_PATH = f"data/hospitals/"

COM_INSIGHT_URL = 'https://www.communitybenefitinsight.org/api/get_hospitals.php'

COST_REPORT_PATH = './data/cost_report/'

TAX_YEAR = 2021
MIN_HOSPITAL_REV = 250_000_000
MAX_HOSPITAL_REV = 500_000_000

INCENTIVE_PERC_LIMIT = 100


cms_root_path = u.get_cms_data(CMS_URL, CMS_ZIP_PATH, CMS_EXTRACT_PATH, CMS_DATASET_ID)


./load/hospitals/hospitals_10_2022.zip already exists. Skipping download
unzipping ./load/hospitals/hospitals_10_2022.zip to data/hospitals/...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:01<00:00, 47.88it/s]


In [8]:

from typing import TypedDict, Literal

class Opts(TypedDict):
    region_pivot: Literal['None', 'Region', 'Division']
    healthcare_system_pivot: bool
    healthcare_system_pivot_small_system_size_cutoff: int
        
def join_datasets(tax_year,
                  min_hospital_rev,
                  max_hospital_rev,
                  cms_root_path,
                  cost_report_path,
                  columns = None,
                  opts: Opts = None
                 ):
    rename_cols = True
    if opts is None:
        opts: Opts = {}  

    if columns is None:
        columns = {}
        rename_cols = False

    _created_dummy_cols = set()
    additional_tables = {}
    
    def _get_dummies_and_record(df: pd.DataFrame, cols: list, **kwargs) -> pd.DataFrame:
        before = set(df.columns)
        df_after = pd.get_dummies(df, columns=cols, **kwargs)
        added = set(df_after.columns) - before
        _created_dummy_cols.update(added)
        return df_after
        
    print(opts)
    
    hosp_info = u.get_hosp_gen_info(cms_root_path)
    hosp_info = hosp_info[['Facility Name', 'Facility ID', 'State']]
    hosp_info['Facility ID'] = hosp_info['Facility ID'].astype(str)
    hosp_info['name_lower'] = hosp_info['Facility Name'].str.lower()

    reg_df = pd.read_csv('state_to_region_map.csv')
    
    def _state_to_field(row, field):
        assert field in ['Region', 'Division']
        res = reg_df[reg_df['State Code'].str.lower() == row['State'].lower()][field]
        if len(res) > 0:
            return res.values[0]
        else:
            return f'{field} Not Found'

    def _state_to_region(row):
        return _state_to_field(row, 'Region')  

        
    def _state_to_division(row):
        return _state_to_field(row, 'Division')   
        

  
    match opts.get('region_pivot', 'None'):
        case 'Region':
            hosp_info['Region'] = hosp_info.apply(_state_to_region, axis=1)
            hosp_info['Region Unencoded'] = hosp_info['Region']
            hosp_info = _get_dummies_and_record(hosp_info, cols=['Region'], drop_first=True, dtype=int)
        case 'Division':
            hosp_info['Region'] = hosp_info.apply(_state_to_division, axis=1)
            hosp_info['Region Unencoded'] = hosp_info['Region']
            hosp_info = _get_dummies_and_record(hosp_info, cols=['Region'], drop_first=True, dtype=int)
        case _:
            pass

    if opts.get('healthcare_system_pivot'):
        # validate opts
        small_cutoff = opts.get('healthcare_system_pivot_small_system_size_cutoff')
        bins_opt = opts.get('healthcare_system_bins')
        if (not small_cutoff and not bins_opt) or (small_cutoff and bins_opt):
            raise ValueError(
                "if healthcare_system_pivot is true, then you must pass an int for "
                "healthcare_system_pivot_small_system_size_cutoff OR a list of ints for healthcare_system_bins (but not both)"
            )

        sys_df = u.get_healthcare_system()
        sys_df['health_sys_id'] = sys_df['health_sys_id'].fillna('Not in a Hospital System')
        sys_df_vals = sys_df['health_sys_id'].value_counts()

        if small_cutoff:
            # collapse small systems into 'Small Hospital System' category
            sys_df['health_sys_id'] = sys_df['health_sys_id'].apply(
                lambda x: 'Small Hospital System' if sys_df_vals.get(x, 0) < small_cutoff else x
            )
            sys_df = sys_df[['ccn', 'health_sys_id']].copy()
            sys_df = _get_dummies_and_record(sys_df, cols=['health_sys_id'], drop_first=True, dtype=int)
        else:
            bin_list = list(opts.get('healthcare_system_bins')) #+ [200, 5000]
            # get value counts per id, create bins
            counts = sys_df['health_sys_id'].value_counts()
                # binned = pd.cut(counts, bins=bin_list)
            # convert to string series index by health_sys_id
                # binned = pd.series(binned.astype(str), index=counts.index)
            # special-case the 'not in a hospital system' label: keep as-is
            # Build mapping from id -> bin label
            # sys_df['health_system_size'] = sys_df['health_sys_id'].apply(lambda x: binned.get(x, 'Not in a Hospital System'))
            sys_df['health_system_size'] = sys_df['health_sys_id'].apply(lambda x: counts.get(x, 0) if x != 'Not in a Hospital System' else 0)
            sys_df = sys_df[['ccn', 'health_system_size', 'health_sys_id']].copy()
            # return sys_df
            # sys_df = _get_dummies_and_record(sys_df, cols=['health_system_size'], dtype=int)

        # merge into hosp_info; ensure the ccns are strings
        sys_df = sys_df.rename(columns=lambda c: str(c))  # defensive
        hosp_info = pd.merge(hosp_info, sys_df, left_on='Facility ID', right_on='ccn', how='left')
        
    bridge = u.get_bridge_file()
    bridge['name_lower'] = bridge['name'].str.lower().fillna('')
    bridge['ein'] = bridge['ein'].astype(str)
    bridge.columns = bridge.columns.map(lambda x: str(x) + " - bridge")
    
    hosp_info = pd.merge(hosp_info,
                         bridge,
                         left_on = 'name_lower',
                         right_on = 'name_lower - bridge',
                         how = 'left')
    
    form_990 = pd.read_csv('form_990_processed.csv')
    df_990_cleaned = u.clean_990(form_990, tax_year, min_hospital_rev, max_hospital_rev)
    df_990_cleaned['ein'] = df_990_cleaned['ein'].astype(str)
    df_990_cleaned.columns = df_990_cleaned.columns.map(lambda x: str(x) + " - 990")

    
    hosp_info = pd.merge(hosp_info,
                         df_990_cleaned,
                         left_on=['ein - bridge', 'State'],
                         right_on = ['ein - 990', 'state - 990'])

    hosp_info = hosp_info[hosp_info['incentive_perc - 990'] <= INCENTIVE_PERC_LIMIT]
    
    
    hcahps = u.get_hcahps(cms_root_path)
    hcahps['name_lower'] = hcahps['Facility Name'].str.lower().fillna('')
    hcahps['Facility ID'] = hcahps['Facility ID'].astype(str)
    hcahps.columns = hcahps.columns.map(lambda x: str(x) + " - HCAHPS")
    hosp_info = pd.merge(hosp_info,
                         hcahps,
                         left_on = 'Facility ID',
                         right_on = 'Facility ID - HCAHPS',
                         how = 'left')

    
    comp_mort = u.get_comp_mort(cms_root_path)
    comp_mort['Facility ID'] = comp_mort['Facility ID'].astype(str)
    comp_mort.columns = comp_mort.columns.map(lambda x: str(x) + " - complications and mortality report")
    hosp_info = pd.merge(hosp_info,
                      comp_mort,
                      right_on = 'Facility ID - complications and mortality report',
                      left_on = 'Facility ID',
                      how = 'left')

    
    cost_report = u.get_cost_report(cost_report_path)
    cost_report['ccn'] = cost_report['Provider CCN'].astype(str)
    cost_report['is_teaching_hospital'] = (cost_report['Number of Interns and Residents (FTE)'] > 0).astype(int)
    cost_report.columns = cost_report.columns.map(lambda x: str(x) + " - cost report")
    hosp_info = pd.merge(hosp_info,
                         cost_report,
                         left_on = 'ccn',
                         right_on = 'ccn - cost report',
                         )
    
    
    merged = hosp_info.copy()

    # Build filename using the function arguments (not undefined globals)
    fname = f'merged_dataset_year={tax_year}_cmsYear=2022_minRev={min_hospital_rev}_maxRev={max_hospital_rev}'

    # Save raw merged
    merged.to_csv(f'{fname}_raw.csv', index=False)

    # If user provided a columns mapping, make sure to keep the dummy columns we created.
    # We default to mapping dummy columns to themselves (so they survive selection).
    if columns is not None:
        # if columns maps original multi-source names to friendly names, we want dummies preserved.
        for dummy_col in sorted(_created_dummy_cols):
            if dummy_col not in columns:
                # preserve it with identity mapping
                columns[dummy_col] = dummy_col

        # rename columns according to mapping (only affects names present)
        merged = merged.rename(columns=columns)

    # If columns is None (rename_cols False), just write full CSV and return the merged raw.
    if not rename_cols:
        merged.to_csv(f'{fname}.csv', index=False)
        return merged

    # else, select only the desired columns (the values of the mapping are the desired output column names)
    desired_cols = list(columns.values())
    # keep only requested columns that actually exist (defensive)
    existing_desired_cols = [c for c in desired_cols if c in merged.columns]
    missing = set(desired_cols) - set(existing_desired_cols)
    if missing:
        # warn user (do not fail) — printing is fine in your environment
        print(f"Warning: requested columns missing from merged dataframe and will be skipped: {sorted(list(missing))}")

    reduced = merged[existing_desired_cols].copy()
    reduced.to_csv(f'{fname}.csv', index=False)
    return reduced
    # hvbp_clin_out = get_hvbp_clinical_outcomes(cms_root_path)

columns = {
    'Facility ID - HCAHPS': 'Facility ID',
    'Facility Name - HCAHPS': 'Facility Name',
    'Region Unencoded': 'Region',
    'health_sys_id': 'health_sys_id',
    'health_system_size': 'health_system_size',
    'ein - 990': "EIN",
    'is_teaching_hospital - cost report': 'is_teaching_hospital',
    'incentive_perc - 990': 'incentive_perc',
    'Number of Beds - cost report': 'Number of Beds',
    'op_margin - 990': 'Operating Margin',

    'Overall hospital rating - linear mean score: HCAHPS Linear Mean Value - HCAHPS': 'Overall Linear Mean HCAHPS Hospital Rating',
    "Death rate for CABG surgery patients: Score - complications and mortality report": "CABG Death Rate Score",
    "Death rate for COPD patients: Score - complications and mortality report": "COPD Death Rate Score",
    "Death rate for heart attack patients: Score - complications and mortality report": "Heart Attack Death Rate Score",
    "Death rate for heart failure patients: Score - complications and mortality report": "Heart Failure Death Rate Score",
    "Death rate for stroke patients: Score - complications and mortality report": "Stroke Death Rate Score",
    "Rate of complications for hip/knee replacement patients: Score - complications and mortality report": "Rate of Complications of Hip or Knee Replacement Score",
    
    
}

bins = [0, 1, 2, 3, 4, 5]
merged_2020 = join_datasets(2020, 
                       MIN_HOSPITAL_REV,
                       MAX_HOSPITAL_REV,
                       cms_root_path,
                       COST_REPORT_PATH,
                       # columns = columns,
                       opts = {
                           'region_pivot': 'Region',
                           'healthcare_system_pivot': True,
                           # 'healthcare_system_pivot_small_system_size_cutoff': 20,
                           'healthcare_system_bins': bins
                       })
# bins += [200, 5000]

merged_2021 = join_datasets(2021, 
                       MIN_HOSPITAL_REV,
                       MAX_HOSPITAL_REV,
                       cms_root_path,
                       COST_REPORT_PATH,
                       # columns = columns,
                       opts = {
                           'region_pivot': 'Region',
                           'healthcare_system_pivot': True,
                           # 'healthcare_system_pivot_small_system_size_cutoff': 20,
                           'healthcare_system_bins': bins
                       })
# bins += [200, 5000]


merged_2022 = join_datasets(2022, 
                       MIN_HOSPITAL_REV,
                       MAX_HOSPITAL_REV,
                       cms_root_path,
                       COST_REPORT_PATH,
                       # columns = columns,
                       opts = {
                           'region_pivot': 'Region',
                           'healthcare_system_pivot': True,
                           # 'healthcare_system_pivot_small_system_size_cutoff': 20,
                           'healthcare_system_bins': bins
                       })
# bins += [200, 5000]
print("num records = ", len(merged_2020))
merged_2020.head()

{'region_pivot': 'Region', 'healthcare_system_pivot': True, 'healthcare_system_bins': [0, 1, 2, 3, 4, 5]}


/home/jovyan/project_utils.py:407: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  hcahps = pd.read_csv(os.path.join(cms_root_path, 'HCAHPS-Hospital.csv'))


{'region_pivot': 'Region', 'healthcare_system_pivot': True, 'healthcare_system_bins': [0, 1, 2, 3, 4, 5]}


/home/jovyan/project_utils.py:407: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  hcahps = pd.read_csv(os.path.join(cms_root_path, 'HCAHPS-Hospital.csv'))


{'region_pivot': 'Region', 'healthcare_system_pivot': True, 'healthcare_system_bins': [0, 1, 2, 3, 4, 5]}


/home/jovyan/project_utils.py:407: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  hcahps = pd.read_csv(os.path.join(cms_root_path, 'HCAHPS-Hospital.csv'))


num records =  52


,Facility Name,Facility ID,State,name_lower,Region Unencoded,Region_Northeast,Region_Region Not Found,Region_South,Region_West,ccn,...,Total Income - cost report,Total Other Expenses - cost report,Net Income - cost report,Cost To Charge Ratio - cost report,Net Revenue from Medicaid - cost report,Medicaid Charges - cost report,Net Revenue from Stand-Alone CHIP - cost report,Stand-Alone CHIP Charges - cost report,ccn - cost report,is_teaching_hospital - cost report
0,STEPHENS COUNTY HOSPITAL,110032,GA,stephens county hospital,South,0,0,1,0,110032,...,8859077.0,15795378.0,-6936301.0,0.461457,2163615.0,7108192.0,23259.0,75900.0,110032,0
1,TAYLOR REGIONAL HOSPITAL,110135,GA,taylor regional hospital,South,0,0,1,0,110135,...,369165.0,NaN,369165.0,0.465105,1388860.0,4863109.0,28833.0,127594.0,110135,0
2,BACON COUNTY HOSPITAL,111327,GA,bacon county hospital,South,0,0,1,0,111327,...,1473733.0,NaN,1473733.0,0.385113,2353806.0,10107483.0,17788.0,100084.0,111327,0
3,NORTH CANYON MEDICAL CENTER,131302,ID,north canyon medical center,West,0,0,0,1,131302,...,4612848.0,2542763.0,2070085.0,0.563468,8572837.0,15296587.0,NaN,NaN,131302,0
4,ROSELAND COMMUNITY HOSPITAL,140068,IL,roseland community hospital,Midwest,0,0,0,0,140068,...,6271549.0,97638.0,6173911.0,0.453773,36518353.0,68851056.0,NaN,NaN,140068,0


In [28]:
for year, merged in zip([2020, 2021, 2022], [merged_2020, merged_2021, merged_2022]):

    merged_hcahps = merged[[c for c in merged.columns if ' - HCAHPS' in c]]
    merged_hcahps.drop(columns = ['Facility ID - HCAHPS', 
                                  'ZIP Code - HCAHPS',
                                  'Facility Name - HCAHPS',
                                  'name_lower - HCAHPS',
                                  'State - HCAHPS',
                                  'City - HCAHPS'], inplace = True)
    merged_hcahps_counts = merged_hcahps.count().sort_values(ascending = False)
    merged_hcahps_variances = merged_hcahps.apply(np.std)

    merged_hcahps_analytics = pd.DataFrame({'index': merged_hcahps_counts.index,
                                            'counts': merged_hcahps_counts.values,
                                            'std_dev': merged_hcahps_variances.values,
                                           })

    merged_hcahps_analytics.to_csv(f'hcahps_{year}_counts_and_std_devs.csv')
                                            
                                            



/tmp/ipykernel_745/354679760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_hcahps.drop(columns = ['Facility ID - HCAHPS',
/tmp/ipykernel_745/354679760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_hcahps.drop(columns = ['Facility ID - HCAHPS',
/tmp/ipykernel_745/354679760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_hcahps.drop(columns = ['Facility ID - HCAHPS',


In [27]:
merged_hcahps_analytics

,index,counts,std_dev
0,"Patients who reported that YES, they were give...",44,5.185012
1,"Patients who reported that their doctors ""Usua...",44,5.382901
2,"Patients who reported that they ""Always"" recei...",44,4.908459
3,Patients who reported that their room and bath...,44,4.784347
4,"Patients who reported that their doctors ""Alwa...",44,4.845925
...,...,...,...
88,Discharge information - linear mean score: HCA...,27,0.948611
89,Communication about medicines - linear mean sc...,27,0.957248
90,Cleanliness - linear mean score: HCAHPS Linear...,27,1.011593
91,Care transition - linear mean score: HCAHPS Li...,27,0.902671
